In [2]:
import pandas as ps
import numpy as np

In [5]:
ecom_txns = ps.read_csv('D:\Data Science\Projects\Fraud Prevention\data\ecom_txns.csv')

In [6]:
ecom_txns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 11 columns):
user_id            150000 non-null int64
signup_datetime    150000 non-null object
datetime           150000 non-null object
amount             150000 non-null float64
device_id          150000 non-null int64
store              150000 non-null object
browser            150000 non-null object
sex                150000 non-null object
age                150000 non-null int64
ip_address         150000 non-null object
fraud              150000 non-null int64
dtypes: float64(1), int64(4), object(6)
memory usage: 12.6+ MB


In [7]:
ip_mappings = ps.read_csv('D:\Data Science\Projects\Fraud Prevention\data\ip_mappings.csv')

In [20]:
ip_mappings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421239 entries, 0 to 421238
Data columns (total 3 columns):
range_start    421239 non-null object
range_end      421239 non-null object
country        420896 non-null object
dtypes: object(3)
memory usage: 9.6+ MB


In [22]:
ecom_txns.head()

,user_id,signup_datetime,datetime,amount,device_id,store,browser,sex,age,ip_address,fraud,int_ip
0,40630765,2016-03-22 09:11:01,2016-06-01 01:58:07,12.05,41974320,toys,IE,F,27,233.235.54.214,0,-370460970
1,90517585,2016-05-17 14:45:10,2016-06-03 13:49:55,55.68,70879186,babies,IE,F,31,144.255.229.236,0,-1862277652
2,63383012,2016-09-02 14:15:02,2016-12-03 13:38:24,33.45,40697637,pets,Chrome,F,49,124.207.12.230,0,2093944038
3,34695055,2016-08-03 01:51:47,2016-11-11 12:49:45,40.32,29544115,toys,FireFox,M,41,4.162.99.116,0,77751156
4,92152176,2016-07-17 04:58:30,2016-08-24 19:12:41,49.16,80967980,babies,Chrome,Not Provided,44,90.168.43.218,0,1520970714


In [23]:
ip_mappings.head()

,range_start,range_end,country,int_range_start,int_range_end
0,0.0.0.0,0.255.255.255,ZZ,0,16777215
1,1.0.0.0,1.0.0.255,AU,16777216,16777471
2,1.0.1.0,1.0.3.255,CN,16777472,16778239
3,1.0.4.0,1.0.7.255,AU,16778240,16779263
4,1.0.8.0,1.0.15.255,CN,16779264,16781311


function to convert ip to numeric representation one at a time

In [11]:
def ip_to_int(ip_ser):
    ips = ip_ser.str.split('.', expand=True).astype(np.int16).values
    mults = np.tile(np.array([24, 16, 8, 0]), len(ip_ser)).reshape(ips.shape)
    return np.sum(np.left_shift(ips, mults), axis=1)

In [19]:
ip_mappings = ip_mappings.head(421239) # remove observations with bad ip addresses

create new column by converting all ip's with numeric representation

In [21]:
ecom_txns['int_ip'] = ip_to_int(ecom_txns.ip_address)
ip_mappings['int_range_start'] = ip_to_int(ip_mappings.range_start)
ip_mappings['int_range_end'] = ip_to_int(ip_mappings.range_end)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


add new column 'index' to ecom_txns, which will contain an index of the first matching IP interval from the ip_mappings

In [24]:
ecom_txns['index'] = (ecom_txns.int_ip.apply(lambda x: ip_mappings.query('int_range_start <= @x <= int_range_end').index.values).apply(lambda x: x[0] if len(x) else -1))

KeyboardInterrupt: 

now merge two data frames

In [ ]:
merged_ecom_txns = (pd.merge(ecom_txns.drop('int_ip',1),
   ip_mappings.filter(regex=r'^((?!.?ip_range_).*)$'),
   left_on='index',
   right_index=True,
   how='left')
   .drop('index',1)
   )